In [ ]:
from keras import backend as K
import os
import json
import numpy as np
import matplotlib.pyplot as plt

%run ../Environment.ipynb  # import Environment
%run ../Agent.ipynb  # import Agent

In [ ]:
# Assuming you have the EdgeComputingEnvironment and DDQNAgent defined as per your code
env = EdgeComputingEnvironment()

# Define the bandwidth values to test (in Hz)
bandwidth_values = [2.5, 5, 7.5, 10, 15]  # Example values in Hz

# Define the number of users/devices
num_users = env.M

# Initialize the DDQN agent
agent = DDQNAgent(env, num_users)

# Define the number of episodes for training and testing steps
num_train_episodes = 1  # or any suitable number for training
num_test_steps = 1       # or any suitable number for testing

# Create the folder for DDQN model files if it doesn't exist
model_folder = 'DDQN_Models_Test_Bandwidth'
os.makedirs(model_folder, exist_ok=True)

# Storage for results
results = []

# Train and save DDQN models for different bandwidth values
for bandwidth in bandwidth_values:
    env.B = bandwidth * 1e6  # Set the bandwidth in the environment
    print(env.B)
    # Train the agent
    agent.train(num_train_episodes)

    # Save the model for this configuration
    model_filename = os.path.join(model_folder, f'DDQN_model_Bandwidth_{int(bandwidth)}MHz.h5')
    agent.save(model_filename)

    # Test and collect metrics
    avg_delay, avg_alpha = agent.test(num_test_steps)

    # Store the results
    results.append({
        'bandwidth': bandwidth,
        'avg_alpha': avg_alpha
    })

# Save the results to a file
with open('DDQN_Bandwidth_results.json', 'w') as f:
    json.dump(results, f)

# Extract the alpha values for plotting
alpha_values = [result['avg_alpha'] for result in results]
bandwidth_labels = [f"{int(bw)} MHz" for bw in bandwidth_values]

# Plotting the results
plt.figure(figsize=(10, 6))

plt.plot(bandwidth_labels, alpha_values, marker='o', linestyle='-', color='blue', label='Average Alpha (Rate of Data Transfer)')
plt.xlabel('Bandwidth')
plt.ylabel('Average Alpha (Rate of Data Transfer)')
plt.title('Impact of Bandwidth on Data Transfer Rate (Alpha)')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()
